In [2]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import applications


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\heber\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-23d902bbc8cf>", line 3, in <module>
    from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
  File "C:\Users\heber\anaconda3\lib\site-packages\tensorflow_core\python\keras\__init__.py", line 27, in <module>
    from tensorflow.python.keras import applications
  File "C:\Users\heber\anaconda3\lib\site-packages\tensorflow_core\python\keras\applications\__init__.py", line 25, in <module>
    from tensorflow.python.keras import engine
  File "C:\Users\heber\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\__init__.py", line 23, in <module>
    from tensorflow.python.keras.engine.base_layer import Layer
  File "C:\Users\heber\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py", line 56, in <module>
    from 

ImportError: cannot import name 'Layer' from 'tensorflow.python.keras.engine.base_layer' (C:\Users\heber\anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py)

In [ ]:
vgg=applications.vgg16.VGG16()

In [ ]:
vgg.summary()

In [ ]:
cnn=Sequential()
for capa in vgg.layers:
    cnn.add(capa)

In [ ]:
cnn.summary()

In [ ]:
cnn.pop()

In [ ]:
cnn.summary()

In [ ]:
for layer in cnn.layers:
    layer.trainable=False

In [ ]:
cnn.add(Dense(3,activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
def modelo():
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
        for capa in vgg.layers:
            cnn.add(capa)
        cnn.layers.pop()
        for layer in cnn.layers:
            layer.trainable=False
        cnn.add(Dense(3,activation='softmax'))
    
    return cnn

In [ ]:
K.clear_session()



data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'


epocas=20
longitud, altura = 150, 150
batch_size = 32
pasos = 1000
validation_steps = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3
lr = 0.0004


##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

##TODO ESTO ES SUSTITUIDO POR LA FUNCION QUE CREA LA RED VGG16
'''cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))
'''


##CREAR LA RED VGG16

cnn=modelo()

cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])


cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')